## T맵에서 제공하는 데이터를 가지고 핫한 여행지를 분석해 보기
1. 데이터를 가져와서 cleaning작업을 해줌(의미있는 데이터만 뽑아오고 나머지는 삭제)
2. 의미있는 데이터에 맨 마지막 열을 보면 검색지랭킹이 존재함
3. 이 검색지 랭킹을 가지고 요일별 평균을 내고 최종적으로 가장 인기있던 지역을 뽑아봅니다.
4. 마지막으로 지도에 표시해봅니다.

In [1]:
# 데이터 가져오기
import pandas as pd
import numpy as np

raw_log = pd.read_csv("Tmap_2008.csv")
raw_log.head()

일자 지역(시도) 지역(시군구)        검색지명 검색지유형1 검색지유형2  검색지유형3  검색지랭킹(시군구내)
0  20200829     강원     고성군      아이파크콘도  여행/레저     숙박  콘도/리조트            1
1  20200829     강원     고성군      소노캄델피노  여행/레저     숙박  콘도/리조트            2
2  20200829     강원     고성군       아야진해변  여행/레저   관광명소    해수욕장            3
3  20200829     강원     고성군  켄싱턴리조트설악비치  여행/레저     숙박  콘도/리조트            4
4  20200829     강원     고성군     고성통일전망대  여행/레저   관광명소     전망대            5

In [2]:
# 의미있는 데이터(제주) 만 남기기 (cleaning)
def clean(location):
    #print(location)
    
    if location == "제주":
        return location
    else:
        return np.nan

raw_log["지역(시도)"] = raw_log["지역(시도)"].apply(clean)
raw_log = raw_log.dropna(axis=0,how="any")

In [3]:
print(raw_log.shape[0])
raw_log

1860


일자 지역(시도) 지역(시군구)       검색지명 검색지유형1  검색지유형2   검색지유형3  \
2070    20200829     제주    서귀포시  서귀포매일올레시장     쇼핑      시장     재래시장   
2071    20200829     제주    서귀포시    오설록티뮤지엄   생활편의  문화생활시설  박물관/기념관   
2072    20200829     제주    서귀포시        쇠소깍  여행/레저    관광명소   관광명소기타   
2073    20200829     제주    서귀포시       신라호텔  여행/레저      숙박       호텔   
2074    20200829     제주    서귀포시      천지연폭포  여행/레저    관광명소    폭포/계곡   
...          ...    ...     ...        ...    ...     ...      ...   
227665  20200826     제주     제주시   호텔시리우스제주  여행/레저      숙박       호텔   
227666  20200826     제주     제주시     마레보리조트  여행/레저      숙박   콘도/리조트   
227667  20200826     제주     제주시     함덕해수욕장  여행/레저    관광명소     해수욕장   
227668  20200826     제주     제주시  오션스위츠제주호텔  여행/레저      숙박       호텔   
227669  20200826     제주     제주시     협재해수욕장  여행/레저    관광명소     해수욕장   

        검색지랭킹(시군구내)  
2070              1  
2071              2  
2072              3  
2073              4  
2074              5  
...             ...  
227665           26  
227666           27  
227667           28  
227668           29  
227669           30  

[1860 rows x 8 columns]

In [4]:
# 의미있는 데이터만(검색지유형2-관광명소 / 검색지유형3에서는 숙소를 제외하기) 남기기
def category_clean(data):
    if data == "관광명소":
        return data
    else:
        return np.nan
    
def detail_category_clean(d):
    if d == "숙소":
        return np.nan
    else:
        return d
    
raw_log["검색지유형2"] = raw_log["검색지유형2"].apply(category_clean)
raw_log = raw_log.dropna(axis=0,how="any")
raw_log["검색지유형3"] = raw_log["검색지유형3"].apply(detail_category_clean)
raw_log = raw_log.dropna(axis=0,how="any")

raw_log.head(20)

<ipython-input-4-8ddb77d12dcb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_log["검색지유형3"] = raw_log["검색지유형3"].apply(detail_category_clean)


일자 지역(시도) 지역(시군구)      검색지명 검색지유형1 검색지유형2  검색지유형3  검색지랭킹(시군구내)
2072  20200829     제주    서귀포시       쇠소깍  여행/레저   관광명소  관광명소기타            3
2074  20200829     제주    서귀포시     천지연폭포  여행/레저   관광명소   폭포/계곡            5
2076  20200829     제주    서귀포시     주상절리대  여행/레저   관광명소  관광명소기타            7
2077  20200829     제주    서귀포시      섭지코지  여행/레저   관광명소  관광명소기타            8
2079  20200829     제주    서귀포시     성산일출봉  여행/레저   관광명소      명산           10
2082  20200829     제주    서귀포시  아쿠아플라넷제주  여행/레저   관광명소     수족관           13
2088  20200829     제주    서귀포시    표선해수욕장  여행/레저   관광명소    해수욕장           19
2091  20200829     제주    서귀포시       선녀탕  여행/레저   관광명소  관광명소기타           22
2096  20200829     제주    서귀포시      원앙폭포  여행/레저   관광명소   폭포/계곡           27
4772  20200821     제주    서귀포시       쇠소깍  여행/레저   관광명소  관광명소기타            3
4774  20200821     제주    서귀포시     천지연폭포  여행/레저   관광명소   폭포/계곡            5
4776  20200821     제주    서귀포시  아쿠아플라넷제주  여행/레저   관광명소     수족관            7
4780  20200821     제주    서귀포시     주상절리대  여행/레저   관광명소  관광명소기타           11
4783  20200821     제주    서귀포시      섭지코지  여행/레저   관광명소  관광명소기타           14
4785  20200821     제주    서귀포시     성산일출봉  여행/레저   관광명소      명산           16
4789  20200821     제주    서귀포시       선녀탕  여행/레저   관광명소  관광명소기타           20
4795  20200821     제주    서귀포시    표선해수욕장  여행/레저   관광명소    해수욕장           26
4796  20200821     제주    서귀포시      원앙폭포  여행/레저   관광명소   폭포/계곡           27
4801  20200821     제주     제주시    협재해수욕장  여행/레저   관광명소    해수욕장            2
4804  20200821     제주     제주시    금능해수욕장  여행/레저   관광명소    해수욕장            5

In [5]:
# 이름은 같고 요일이 다른 데이터를 평균내기

# 우선 "검색지명"을 기준으로 정렬한다
raw_log = raw_log.sort_values(by="검색지명")

raw_log

일자 지역(시도) 지역(시군구)    검색지명 검색지유형1 검색지유형2 검색지유형3  검색지랭킹(시군구내)
168458  20200828     제주     제주시  곽지해수욕장  여행/레저   관광명소   해수욕장            9
128674  20200812     제주     제주시  곽지해수욕장  여행/레저   관광명소   해수욕장            5
155770  20200818     제주     제주시  곽지해수욕장  여행/레저   관광명소   해수욕장           11
79956   20200816     제주     제주시  곽지해수욕장  여행/레저   관광명소   해수욕장            7
126756  20200830     제주     제주시  곽지해수욕장  여행/레저   관광명소   해수욕장            7
...          ...    ...     ...     ...    ...    ...    ...          ...
136139  20200801     제주    서귀포시   황우지해안  여행/레저   관광명소   해수욕장           30
14608   20200803     제주    서귀포시   황우지해안  여행/레저   관광명소   해수욕장           29
186988  20200809     제주    서귀포시   황우지해안  여행/레저   관광명소   해수욕장           29
118528  20200831     제주    서귀포시   황우지해안  여행/레저   관광명소   해수욕장           29
99776   20200802     제주    서귀포시   황우지해안  여행/레저   관광명소   해수욕장           27

[640 rows x 8 columns]

In [6]:
# "검색지랭킹(시군구내)" 데이터를 평균 냅니다.
# 평균 값이 가장 작은 값이 가장 인기있는 장소입니다.


rank = [i for i in raw_log['검색지랭킹(시군구내)']]
place = [j for j in raw_log['검색지명']]


ex_place = raw_log.iloc[0,3]
ave_rank = []
ave_place = []
count = 0
average = 0

for i, j in zip(rank, place):
    if j != ex_place:
        #print(average, count)
        ave_rank.append(average//count)
        ave_place.append(ex_place)
        ex_place = j
        average = i
        count = 1
        #print(average, count)
    else:
        average += i
        count += 1
        
summer = []
for i,j in zip(ave_rank,ave_place):
    summer.append([i,j])
    
    
# 가장 인기있는 여행지 가져오기
summer.sort()
print(summer)

[[3, '쇠소깍'], [3, '협재해수욕장'], [5, '금능해수욕장'], [6, '천지연폭포'], [7, '함덕해수욕장'], [8, '곽지해수욕장'], [8, '아쿠아플라넷제주'], [9, '김녕해수욕장'], [9, '섭지코지'], [10, '주상절리대'], [11, '비자림'], [12, '성산일출봉'], [13, '용두암'], [18, '이호테우해변'], [21, '선녀탕'], [21, '안돌오름'], [21, '표선해수욕장'], [22, '세화해수욕장'], [22, '엉또폭포'], [23, '만장굴'], [24, '금오름'], [25, '산방산탄산온천'], [26, '새별오름'], [26, '원앙폭포'], [27, '월정리해변'], [29, '제주절물자연휴양림'], [29, '천제연폭포'], [30, '카멜리아힐']]


In [7]:
# 겨울 데이터 뽑아오기

raw_log = pd.read_csv("Tmap_2001.csv")

raw_log["지역(시도)"] = raw_log["지역(시도)"].apply(clean)
raw_log = raw_log.dropna(axis=0,how="any")

raw_log["검색지유형2"] = raw_log["검색지유형2"].apply(category_clean)
raw_log = raw_log.dropna(axis=0,how="any")
raw_log["검색지유형3"] = raw_log["검색지유형3"].apply(detail_category_clean)
raw_log = raw_log.dropna(axis=0,how="any")

raw_log = raw_log.sort_values(by="검색지명")

rank = [i for i in raw_log['검색지랭킹(시군구내)']]
place = [j for j in raw_log['검색지명']]


ex_place = raw_log.iloc[0,3]
ave_rank = []
ave_place = []
count = 0
average = 0

for i, j in zip(rank, place):
    if j != ex_place:
        #print(average, count)
        ave_rank.append(average//count)
        ave_place.append(ex_place)
        ex_place = j
        average = i
        count = 1
        #print(average, count)
    else:
        average += i
        count += 1
        
winter = []
for i,j in zip(ave_rank,ave_place):
    winter.append([i,j])
    
    
# 가장 인기있는 여행지 가져오기
winter.sort()
print(winter)

[[2, '협재해변'], [4, '성산일출봉'], [5, '카멜리아힐'], [5, '함덕해수욕장'], [7, '새별오름'], [8, '제주동백수목원'], [9, '산방산탄산온천'], [9, '아쿠아플라넷제주'], [10, '섭지코지'], [11, '쇠소깍'], [11, '엉또폭포'], [11, '용두암'], [12, '비자림'], [14, '천지연폭포'], [17, '주상절리대'], [18, '광치기해변'], [18, '이호테우해변'], [19, '곽지해수욕장'], [19, '에코랜드'], [20, '제주여미지식물원'], [21, '용눈이오름'], [23, '만장굴'], [23, '휴애리동백축제'], [25, '한림공원'], [26, '수목원테마파크'], [26, '용머리해안'], [26, '제주민속촌']]


## 지도에 여름/겨울에 인기있는 관광지 표시하기

In [8]:
import folium

In [9]:
m = folium.Map(location=[33.3943025,126.5426923], zoom_start=10)
m

### 지오코딩으로 경도/위도 뽑기엔 너무 어려워질것 같아서 경도/위도값을 직접 가져옴 - 구글맵 상단 주소창에서 뽑아낼수있음

여름  
쇠소깍 : [33.252166,126.6227275]  
협재해수욕장 : [33.394131, 126.239771]  
금능해수욕장 : [33.3899278,126.2266126]  

겨울  
협재해수욕장 : [33.394131, 126.239771]  
성산일출봉 : [33.4580731,126.9337452]  
카멜리아힐 : [33.2889983,126.3679285]   



In [10]:
# 테스트용
folium.Marker(
    location = [33.252166,126.6227275],
    popup = '쇠소깍',
    icon = folium.Icon(icon='cloud')
).add_to(m)

m

In [11]:
# 지도에 각각 다른색으로 표시하기
# 문제 : 같은위치에 중복해서 마킹을 하면 마지막 마킹만 나오는 것 같습니다
summer_location = {'쇠소깍':[33.252166,126.6227275], '협재해수욕장':[33.394131, 126.239771], '금능해수욕장':[33.3899278,126.2266126]}
winter_location = {'협재해수욕장':[33.394131, 126.239771], '성산일출봉':[33.4580731,126.9337452], '카멜리아힐':[33.2889983,126.3679285]}


def 지도에_마커넣기(딕셔너리, 색):
    for i in 딕셔너리:
        folium.Marker(
        location = 딕셔너리[i],
        popup = i,
        icon = folium.Icon(color=색)
    ).add_to(m)
        

지도에_마커넣기(summer_location,'blue')
지도에_마커넣기(winter_location,'red')
m